<a href="https://colab.research.google.com/github/alvinsbkt/logistics-performance-data-analytics/blob/main/Data_Analytics_Logistics_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It is a Data analysis competition organized by shopee with the problem statement can be accessed [here](https://www.kaggle.com/c/logistics-shopee-code-league/overview)

#Preparation

In [ ]:
!pip install kaggle #installation module kaggle package
!apt-get install -y fuse-zip #installation fusezip to unzip dataset

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "***insuba***" #use your own kaggle username and API key
os.environ['KAGGLE_KEY'] = "*****c44ee6619a3931737cfe25*****" #hidden

In [ ]:
!kaggle competitions download -c logistics-shopee-code-league #download data

  0% 0.00/9.01k [00:00<?, ?B/s]
100% 9.01k/9.01k [00:00<00:00, 10.1MB/s]
 99% 379M/381M [00:04<00:00, 102MB/s] 
100% 381M/381M [00:04<00:00, 90.2MB/s]


In [ ]:
input_dir = "/tmp/kaggle-data"
!mkdir {input_dir}
!fuse-zip /content/delivery_orders_march.csv.zip {input_dir} #unzipping dataset file from kaggle
!ls {input_dir} #checking unzip folder result

delivery_orders_march.csv


In [ ]:
import pandas as pd
import time
import math
from datetime import date, timedelta
import datetime

In [ ]:
df=pd.read_csv("/tmp/kaggle-data/delivery_orders_march.csv") #get unzipped dataset to notebook

#Getting Time
obtaining readable and proccesable time data from the dataset and put it to a new column

In [ ]:
def gettime(x):
  if math.isnan(x):
    return x
  else:
    return time.gmtime(x).tm_yday

df['first_time']=df['1st_deliver_attempt'].apply(lambda x: time.gmtime(x).tm_yday)
df['second_time']=df['2nd_deliver_attempt'].apply(lambda x: gettime(x))

#Getting Location
Obtaining readable and proccesable data of location of both buyers and sellers, it is done by annotating each location with a unique integer

There are 4 distinct locations for both the address of buyers and sellers. There are Metro Manila, Luzon, Visayas, and Mindana. Each locations will be annotated with 1,2,3, and 4 respectively.

Then the Service Level Agreement which depends on the sellers address and buyers address are determined and put in a new column

In [ ]:
#getting location of buyers
df[df['buyeraddress'].str.contains("metro manila",case=False)].index
df.loc[df['buyeraddress'].str.contains("metro manila",case=False),'buyer']=1
df.loc[df['buyeraddress'].str.contains("luzon",case=False),'buyer']=2
df.loc[df['buyeraddress'].str.contains("visayas",case=False),'buyer']=3
df.loc[df['buyeraddress'].str.contains("mindanao",case=False),'buyer']=4

In [ ]:
df['buyer'].value_counts()

1.0    1559099
2.0     805484
3.0     479986
4.0     331744
Name: buyer, dtype: int64

In [ ]:
#getting location of sellers
df.loc[df['selleraddress'].str.contains("metro manila",case=False),'seller']=1
df.loc[df['selleraddress'].str.contains("luzon",case=False),'seller']=2
df.loc[df['selleraddress'].str.contains("visayas",case=False),'seller']=3
df.loc[df['selleraddress'].str.contains("mindanao",case=False),'seller']=4

In [ ]:
df['seller'].value_counts() #turned out there are only two distinct locations for sellers

1.0    3173960
2.0       2353
Name: seller, dtype: int64

In [ ]:
def getsla(x):
  if (x[0]==1 and x[1]==1):
    return 3
  elif (x[0]==2 and x[1]==1) or (x[0]==2 and x[1]==2) or (x[0]==1 and x[1]==2):
    return 5
  else:
    return 7

df['sla']=df[['buyer','seller']].apply(lambda x:getsla(x),axis=1) #getting the SLA for every row

#Counting Logistic Days

First we need to check the day of the year of sundays within the range of the logistic, also the day of the year where it is a holiday

In [ ]:
#to check the dates which are sundays
def allsundays(year):
   d = date(year, 3, 1)                    # March 1st
   d += timedelta(days = 6 - d.weekday())  # First Sunday
   while d.year == year and d.month<=4:
      yield d
      d += timedelta(days = 7)

for d in allsundays(2020):
   print(d)

2020-03-01
2020-03-08
2020-03-15
2020-03-22
2020-03-29
2020-04-05
2020-04-12
2020-04-19
2020-04-26


The weekends (sunday) are:

2020-03-01

2020-03-08

2020-03-15

2020-03-22

2020-03-29

2020-04-05 (still within the range)

With the following holidays:

2020-03-08 (Sunday);

2020-03-25 (Wednesday);

2020-03-30 (Monday);

2020-03-31 (Tuesday)


In [ ]:
#used to check the day in the year for all of the sundays and holidays
timestamp = datetime.datetime(2020, 4, 5, 0, 0).timestamp()
print(timestamp)
time.gmtime(timestamp).tm_yday

1586044800.0


96

In [ ]:
exclude=[61,68,75,82,85,89,90,91,96] #list filled with days that the logistics do not operate

61

In [ ]:
#defined a function to annotate whether the package is late or not given the informations provided
def getdif(x):
  a=x[1].copy()
  for i in range(len(exclude)):
    if x[2]<=exclude[i]<=x[1]:
      a=a-1    
  if a-x[2]>x[3]:
    return 1
  else:
    if math.isnan(x[0])==False:
      a=x[0].copy()
      for i in range(len(exclude)):
        if x[1]<=exclude[i]<=x[0]:
          a=a-1
      if a-x[1]>3:
        return 1
      else:
        return 0
    else:
      return 0

In [ ]:
df['is_late']=df[['second_time','first_time','pick_time','lsa']].apply(lambda x:getdif(x),axis=1) #made a new column is_late filled with the label

In [ ]:
#saving and exporting
df_save=df[['orderid','is_late']]
df_save.to_csv("/content/submission5_rev.csv",index=False)